In [1]:
import sys
sys.path.append('/home/lisaanne/lib/coco-caption/')
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
import numpy as np
from bias_detection import *
import json

In [3]:
#json file for annotations for entire set
save_file = '/data/lisaanne/fairness/checkpoints/LW_train/result-checkpoint-1000000.json'   
predicted_captions = json.load(open(save_file))

In [4]:
coco = COCO('coco/annotations/captions_val2014.json')
generation_coco = coco.loadRes(save_file)
coco_evaluator = COCOEvalCap(coco, generation_coco)
print "Evaluation over the entire MSCOCO set:"
coco_evaluator.evaluate()

loading annotations into memory...
0:00:00.576215
creating index...
index created!
Loading and preparing results...     
DONE (t=0.13s)
creating index...
index created!
Evaluation over the entire MSCOCO set:
tokenization...
setting up scorers...
computing Bleu score...
{'reflen': 381597, 'guess': [383026, 342522, 302018, 261514], 'testlen': 383026, 'correct': [254819, 121042, 55258, 26936]}
ratio: 1.00374478835
Bleu_1: 0.665
Bleu_2: 0.485
Bleu_3: 0.350
Bleu_4: 0.258
computing METEOR score...
METEOR: 0.228
computing Rouge score...
ROUGE_L: 0.492
computing CIDEr score...
CIDEr: 0.791


In [5]:
print "Evaluation over the biased set:"

#First create json with unbiased data.  
#Kaylee -- you might want to update this with the script you wrote; I made a text file with all the image names
#since this seemed like something I would use a lot (used your script to make this txt file)

bias_ids = open('/data/lisaanne/fairness/data/img_names.txt').readlines()
bias_ids = [int(id) for id in bias_ids]

bias_captions = []
for caption in predicted_captions:
    if caption['image_id'] in bias_ids:
        bias_captions.append(caption)
        
bias_save_file = save_file.replace('.json', '_bias.json')
with open(bias_save_file, 'w') as outfile:
    json.dump(bias_captions, outfile)

generation_coco = coco.loadRes(bias_save_file)
coco_evaluator = COCOEvalCap(coco, generation_coco)

# evaluate on a subset of images by setting
# cocoEval.params['image_id'] = cocoRes.getImgIds()
# please remove this line when evaluating the full validation set
coco_evaluator.params['image_id'] = generation_coco.getImgIds()

print "Evaluation over the biased MSCOCO set:"
coco_evaluator.evaluate()

Evaluation over the biased set:
Loading and preparing results...     
DONE (t=0.06s)
creating index...
index created!
Evaluation over the biased MSCOCO set:
tokenization...
setting up scorers...
computing Bleu score...
{'reflen': 104406, 'guess': [104419, 93496, 82573, 71650], 'testlen': 104419, 'correct': [73957, 38033, 17474, 9229]}
ratio: 1.00012451392
Bleu_1: 0.708
Bleu_2: 0.537
Bleu_3: 0.394
Bleu_4: 0.298
computing METEOR score...
METEOR: 0.255
computing Rouge score...
ROUGE_L: 0.530
computing CIDEr score...
CIDEr: 0.792


In [6]:
bias_ids = [int(id) for id in bias_ids]
bias_ids = set(bias_ids) & set(coco_evaluator.imgToEval.keys())
bleu_3 = np.mean([coco_evaluator.imgToEval[id]["Bleu_3"] for id in bias_ids])
bleu_4 = np.mean([coco_evaluator.imgToEval[id]["Bleu_4"] for id in bias_ids])
cider = np.mean([coco_evaluator.imgToEval[id]["CIDEr"] for id in bias_ids])
meteor = np.mean([coco_evaluator.imgToEval[id]["METEOR"] for id in bias_ids])

print "Bleu_3: %f" %bleu_3
print "Bleu_4: %f" %bleu_4
print "METEOR: %f" %meteor
print "CIDEr: %f" %cider

Bleu_3: 0.303824
Bleu_4: 0.188967
METEOR: 0.271968
CIDEr: 0.791515


In [7]:
#Get F1 scores for man/woman + some eval preprocessing
#From Kaylee's code

img_2_anno_dict = create_dict_from_list(pickle.load(open(target_train)))
img_2_anno_dict.update(create_dict_from_list(pickle.load(open(target_test))))
img_2_anno_dict.update(create_dict_from_list(pickle.load(open(target_val))))

img_2_anno_dict_simple = {}
for key, value in img_2_anno_dict.iteritems():
    id = int(key.split('_')[-1].split('.jpg')[0])
    img_2_anno_dict_simple[id] = {}
    img_2_anno_dict_simple[id]['male'] = value[0]
    img_2_anno_dict_simple[id]['female'] = value[1]        

In [8]:
def accuracy(predicted):
    f_tp = 0.
    f_fp = 0.
    f_tn = 0.
    f_other = 0.
    f_total = 0.
    
    
    m_tp = 0.
    m_fp = 0.
    m_tn = 0.
    m_other = 0.
    m_total = 0.

    for prediction in predicted:
        image_id = prediction['image_id']
        male = img_2_anno_dict_simple[image_id]['male']
        female = img_2_anno_dict_simple[image_id]['female']
        pred_male = classified_as_man(prediction['caption'].split(' '))
        pred_female = classified_as_woman(prediction['caption'].split(' '))
        if (female & pred_female):
            f_tp += 1
        if (male & pred_male):
            m_tp += 1
        if (male & pred_female):
            f_fp += 1
        if (female & pred_male):
            m_fp += 1
        if ((not female) & (not pred_female)):
            f_tn += 1
        if ((not male) & (not pred_male)):
            m_tn += 1
        pred_other = (not pred_male) & (not pred_female)
        if (female & pred_other):
            f_other += 1
        if (male & pred_other):
            m_other += 1
        if female:
            f_total += 1
        if male:
            m_total += 1

        
    #precision_f = (f_tp)/(f_tp+f_fp)
    #recall_f = (f_tp)/(f_tp+f_n)
    
    print "Of female images:"
    print "Woman predicted %f percent." %(f_tp/f_total)
    print "Man predicted %f percent." %(m_fp/f_total)
    print "Other predicted %f percent." %(f_other/f_total)
    
    print "Of male images:"
    print "Man predicted %f percent." %(m_tp/m_total)
    print "Woman predicted %f percent." %(f_fp/m_total)
    print "Other predicted %f percent." %(m_other/m_total)
    #precision_m = (m_tp)/(m_tp+m_fp)

In [9]:
predicted_all = json.load(open(save_file))
predicted = [p for p in predicted_all if p['image_id'] in bias_ids]

In [10]:
accuracy(predicted)

Of female images:
Woman predicted 0.457487 percent.
Man predicted 0.335660 percent.
Other predicted 0.206853 percent.
Of male images:
Man predicted 0.747652 percent.
Woman predicted 0.074379 percent.
Other predicted 0.177969 percent.


In [13]:
np.mean([coco_evaluator.imgToEval[id]["CIDEr"] for id in coco_evaluator.imgToEval.keys()])

0.79101707009731126

In [11]:
bias_captions

[]